<a href="https://colab.research.google.com/github/laibimane/Master_AM_RECH902_2223/blob/main/Analyse_primaire__du_Contenu_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 46.9 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF
import pandas as pd

def analyze_pdf(file_path):
    try:
        doc = fitz.open(file_path)
        total_text_length = 0
        total_image_area = 0

        for page in doc:
            # Extract text
            text = page.get_text()
            total_text_length += len(text)

            # Extract images and calculate total image area
            for img in page.get_images(full=True):
                total_image_area += img[0]

        total_content = total_text_length + total_image_area

        # Calculate percentages
        text_percentage = 0
        image_percentage = 0
        if total_content > 0:
            text_percentage = (total_text_length / total_content) * 100
            image_percentage = (total_image_area / total_content) * 100

        df = pd.DataFrame(columns=['path', 'nombre_de_pages', 'pourcentage_de_texte', 'pourcentage_d_images', 'image_list'])

        # Store results in DataFrame
        df.loc[0] = [file_path, len(doc), text_percentage, image_percentage, page.get_images(full=True)]

        doc.close()
        return df
    except fitz.fitz.FileDataError as e:
        print(f"Error processing {file_path}: {e}")
        return None  # Return None to indicate an error occurred
    except fitz.fitz.FileNotFoundError as e:
        print(f"File not found: {file_path}")
        return None  # Return None to indicate the file is missing

def process_pdf_files(pdf_file_paths):
    result_df = pd.DataFrame(columns=['path', 'nombre_de_pages', 'pourcentage_de_texte', 'pourcentage_d_images', 'image_list'])

    # Analyze each PDF file and accumulate the results in the result_df
    for path in pdf_file_paths:
        df = analyze_pdf(path)
        if df is not None:
            result_df = pd.concat([result_df, df], ignore_index=True)

    return result_df

def main():
    excel_file_path = "C:\\Users\\Imane LAIB\\Documents\\SIAAP\\Liste_pdf.xlsx"  # Replace with the actual path
    df_paths = pd.read_excel(excel_file_path, usecols=['Chemin_final'])
    pdf_file_paths = df_paths['Chemin_final'].tolist()[:500]  # Process only the first 500 values from the list

    result_df = process_pdf_files(pdf_file_paths)

    excel_output_path = 'output_info_new.xlsx'
    result_df.to_excel(excel_output_path, index=False)
    print(f"Combined DataFrame info exported to {excel_output_path}")

if __name__ == "__main__":
    main()
